In [101]:
import pandas as pd
import sqlite3
import numpy as np
import random

importa as libs

In [102]:
conection = sqlite3.connect("./db/db.db")
cursor = conection.cursor()
df = pd.read_sql_query("SELECT * FROM iris", conection, index_col="id")

conecta com o banco de dados

In [103]:
def pontoAleatorio(n:int=2):
    if n<2:
        raise ValueError(f"{n} é menor que 2")
    elif type(n) not in [int,float]:
        raise ValueError(f"{n} não é numerico")
    else:
        return np.array([
        [random.uniform(df[col].min(), df[col].max()) for col in df.columns] 
        for _ in range(n)
    ])

retorna um np.array de um ponto com todas as cordenadas aleatórias

In [104]:
def distanciaPontoAPonto(ponto1:np.array, ponto2:np.array):
    somaquadrados = 0
    for i in range(len(ponto1)):
        somaquadrados += np.square(ponto1[i] - ponto2[i])
    return np.sqrt(somaquadrados)

retorna a distancia entre dois pontos a partida da entrada ponto1 e ponto 2

In [105]:
def relacionaClasseCentroide(n: int,pontos):
    json = {}
    for i in range(len(pontos)):
        json[i + 1] = pontos[i]
    return json


retorna um json com n pontos e suas classes correspondentes a partir de um input numérico n

In [106]:
def classificaPonto(ponto:np.array,centroides:dict):
    classe=1
    menor_dist=None
    for x in centroides.keys():
        if menor_dist==None:
            classe = x
            menor_dist = distanciaPontoAPonto(ponto,centroides[x])
        elif distanciaPontoAPonto(ponto, centroides[x])<menor_dist:
            classe = x
            menor_dist = distanciaPontoAPonto(ponto,centroides[x])
    return classe

a partir de um ponto e o dicionário no formato retornado pela função relacionaClasseCentroide.

In [107]:
def aplica_ao_db(n,pontos):
    centroides=relacionaClasseCentroide(n,pontos)
    colunas_numericas=[col for col in df.columns]
    df['classe'] = df.apply(lambda row: classificaPonto(np.array(row[colunas_numericas]), centroides), axis=1)

calcula a distancia no dataset

In [108]:
def selecionaClasses(n,pontos):
    aplica_ao_db(n,pontos)
    json={}
    for i in range(1,n+1):
        json[i]=df[df['classe'] == i]
    return json


um data base de acordo com cada classe

In [109]:
def selecionaporclasse():
    classes=df['classe'].unique()
    json={}
    for classe in classes:
        json[classe]=df.loc[df['classe']==classe]
    return json

retorna um json com os dados organizados de acordo com cada classe

In [110]:
def recauculaCentroide():
    dados=selecionaporclasse()
    json={}
    classes=dados.keys()
    for classe in classes:
        json[classe]=np.array([df[x].mean() for x in df.columns])
    return json


retorna um json com o seguinte esquema
valores: centroides calculado
chaves: devida classe do centroide

In [111]:
def exibe():
    json=selecionaporclasse()
    texto=''
    for i in range(1,len(json.keys())+1):
        texto+=f"{i}:{len(json[i]['classe'])}\n"
    print(texto)

exibe o numero de dados de cada classe no dataframe